In [1]:
import numpy as np
import torch
import scipy.optimize

np_rng = np.random.default_rng()

/Users/zhangc/miniconda3/envs/metal/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## bi-quadratic optimization problem

Given a $16\times 16\times 16\times 16$ tensor (or say 4-dimensional array) $B$, we want to find the minimum value of the following function

$$\min_{x,y} \sum_{ijkl} x_iy_jB_{ijkl}x_ky_l$$

where $x,y$ are united vector in real space

$$x,y\in\mathbb{R}^{16},||x||_2=||y||_2=1$$

The `hf_loss_function()/hf_loss_function_with_grad()` below just implement the above function.

## bi-quadratic优化问题

给定一个 $16\times 16\times 16\times 16$ 张量 (或者说四维数组) $B$, 目标是找到下面这个函数的最小值

$$\min_{x,y} \sum_{ijkl} x_iy_jB_{ijkl}x_ky_l$$

其中 $x,y$ 是实数域上的16维的单位长度矢量

$$x,y\in\mathbb{R}^{16},||x||_2=||y||_2=1$$

下方的 `hf_loss_function()/hf_loss_function_with_grad()` 正是实现了这个函数

In [2]:
def hf_loss_function(vecX, vecY, matB):
    dimX = vecX.shape[0]
    dimY = vecY.shape[0]
    vecX = vecX / np.linalg.norm(vecX)
    vecY = vecY / np.linalg.norm(vecY)
    ret = np.einsum(matB, [0,1,2,3], vecX, [0], vecY, [1], vecX, [2], vecY, [3], [], optimize=True)
    return ret


def hf_loss_function_with_grad(vecXY, matB):
    # same function but with gradient (useful for optimization)
    torchX = torch.tensor(vecXY[:16], dtype=torch.float64, requires_grad=True)
    torchY = torch.tensor(vecXY[16:], dtype=torch.float64, requires_grad=True)
    torchB = torch.tensor(matB, dtype=torch.float64)
    tmp0 = torchX / torch.linalg.norm(torchX)
    tmp1 = torchY / torch.linalg.norm(torchY)
    fval = torch.einsum(torchB, [0,1,2,3], tmp0, [0], tmp1, [1], tmp0, [2], tmp1, [3], [])
    fval.backward()
    grad = np.concatenate([torchX.grad.detach().numpy(), torchY.grad.detach().numpy()])
    return fval.item(), grad


Actually, we already know its optimal points

$$x_1=x_6=x_{11}=x_{16}=\frac{1}{2}$$

$$y_1=-y_6=y_{11}=-y_{16}=\frac{1}{2}$$

实际上，我们知道这个问题的解析解

$$x_1=x_6=x_{11}=x_{16}=\frac{1}{2}$$

$$y_1=-y_6=y_{11}=-y_{16}=\frac{1}{2}$$

In [3]:
matB = np.load('biquadratic_mat.npy') #(np,float64,(16,16,16,16))

Xstar = np.eye(4).reshape(-1)/2
Ystar = np.diag([1,-1,1,-1]).reshape(-1)/2
print('minimum:', hf_loss_function(Xstar, Ystar, matB))
print('minimum:', hf_loss_function(Ystar, Xstar, matB)) #inter-changable

minimum: -7.703719777548943e-34
minimum: -7.703719777548943e-34


But if we pretend we don't know the answer and want to find it using optimization, we always fail.

但如果我们假装不知道答案，而是尝试通过优化算法来寻找解，那几乎总会失败

In [4]:
Xtest = np_rng.normal(size=16)
Ytest = np_rng.normal(size=16)
print('initial loss:', hf_loss_function(Xtest, Ytest, matB))

theta0 = np.concatenate([Xtest,Ytest])
theta_optim = scipy.optimize.minimize(hf_loss_function_with_grad, theta0, args=matB, method='L-BFGS-B', jac=True, tol=1e-10)
print('converged loss:', theta_optim.fun) #almost always never be 0

initial loss: 1.0074233556485386
converged loss: 0.008879706450784923


Here is the challenge, could you provide a method to find this optimal point.

挑战：对于这个优化问题，你能否提出一个也许可行的解决方案呢